In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

In [ ]:
from tmqrfeed.manager import DataManager
from tmqrfeed.quotes.quote_contfut import QuoteContFut
from tmqrfeed.costs import Costs
from datetime import datetime
import pandas as pd

In [ ]:
dm = DataManager(date_start=datetime(2011, 6, 1))

In [ ]:
dm.series_primary_set(QuoteContFut, 'US.ES',
                      timeframe='D')
dm.costs_set('US', Costs())

In [ ]:
ohlc = dm.quotes()

In [ ]:
dt = datetime(2017, 1, 1)

In [ ]:
ohlc.index[ohlc.index > dt]

In [ ]:
%%cython
cimport cython
import numpy as np
import pandas as pd
cimport numpy as np
DTYPE_float = np.float
ctypedef np.float64_t DTYPE_t_float
ctypedef np.uint64_t DTYPE_t_uint64
ctypedef np.uint8_t DTYPE_t_uint8
from libc.math cimport abs, isnan
import warnings

np.import_array()

def exposure(data,
             np.ndarray[DTYPE_t_uint8, ndim=1, cast=True] entry_rule,
             np.ndarray[DTYPE_t_uint8, ndim=1, cast=True] exit_rule,
             int direction):
    """
    Backtester routine calculate equity based on data['exo'] and entry/exit rules
    :param data: raw data for backtesting
    :param entry_rule: 1/0 array of entry points
    :param exit_rule: 1/0 array of exit points
    :param direction: Direction of trades, 1 - for long, -1 - for shorts
    :return: tuple(pl, inposition)
        pl - profit-loss inside a particular trade
        inposition - 1/0 array indicating whether the EXO is in or out of the market at the end of the day
    """
    cdef np.ndarray[DTYPE_t_float, ndim=1] price = data.values

    cdef int inpos = 0
    cdef int i = 0
    cdef float pnl = 0.0
    cdef float px = 0.0
    cdef int barcount = price.shape[0]


    cdef np.ndarray[DTYPE_t_float, ndim=1] pl = np.zeros(barcount)
    cdef np.ndarray[DTYPE_t_uint8, ndim=1] inpositon = np.zeros(barcount, dtype=np.uint8)

    for i in range(barcount):
        if inpos == 0:
            # We have a signal, let's open position
            if entry_rule[i] == 1:
                inpos = 1
                inpositon[i] = 1
            else:
                inpositon[i] = 0

        else:
            # Calculate pl
            if exit_rule[i] == 1:
                inpos = 0
                inpositon[i] = 0
            else:
                inpositon[i] = 1

    return pd.Series(inpositon, index=data.index)

def calc_costs(float transaction_costs,float rollover_costs, float prev_exp, float current_exp):
    # If rollover occurred
    cdef float _costs_value = 0.0
    if rollover_costs != 0:
        _costs_value += (-abs(rollover_costs) * abs(prev_exp))

    _costs_value += (-abs(transaction_costs) * abs(prev_exp - current_exp))

    return _costs_value


@cython.cdivision(True)
@cython.boundscheck(False)
def score_netprofit(np.ndarray[DTYPE_t_float, ndim=1] price_series, 
                    exposure, 
                    costs=None):
    # Calculate trade-by-trade payoffs
    cdef float profit = 0.0
    cdef int entry_i = -1

    cdef np.ndarray[DTYPE_t_float, ndim=1] _price = price_series

    try:
        _exposure = exposure.values
    except AttributeError:
        _exposure = exposure

    cdef int barcount = _price.shape[0]

    cdef int i = 0
    cdef int v = 0
    cdef float _costs_value = 0.0
    cdef float current_exp = 0.0
    cdef float prev_exp = 0.0

    cdef int has_costs = costs is not None


    cdef np.ndarray[DTYPE_t_float, ndim=1] rollover_costs
    cdef np.ndarray[DTYPE_t_float, ndim=1] transaction_costs

    if has_costs:
        transaction_costs = costs

    for i in range(1, barcount):
        # Calculate cumulative profit inside particular trade
        current_exp = _exposure[i]
        prev_exp = _exposure[i-1]

        profit += (_price[i] - _price[i-1]) * prev_exp

        # Apply transaction costs
        if has_costs:
            _costs_value = calc_costs(transaction_costs[i], 0, prev_exp, current_exp)
            profit += _costs_value

    return profit


In [ ]:
from backtester.analysis import *

class AlphaGeneric:
    def __init__(self, dm):
        self.dm = dm
        
    def calculate(self, *args, **kwargs):
        direction = 1
        period_slow, period_fast = args
       
        # Defining EXO price
        px = self.dm.quotes()['c']

        #
        #
        # Indicator calculation
        #
        #
        slow_ma = px.rolling(period_slow).mean()
        fast_ma = px.rolling(period_fast).mean()

        # Enry/exit rules
        entry_rule = CrossDown(fast_ma, slow_ma)
        exit_rule = (CrossUp(fast_ma, slow_ma))
        
        return exposure(px, entry_rule.values, exit_rule.values, direction)
    
           
        

In [ ]:
a = AlphaGeneric(dm)

In [ ]:
strategy_exposure = a.calculate(10, 5)

In [ ]:
%%timeit
score_netprofit(ohlc['c'].values, strategy_exposure.values)

In [ ]:
import itertools
from collections import OrderedDict

In [ ]:
PARAMS = [
    #('direction', [1, -1]),
    ('period_fast', list(range(1, 50))),
    ('period_slow', list(range(10, 200))),
]

# Brute force optimization

In [ ]:
results = []
for p in params_universe:
    net_profit = a.eval_net_profit(p)
    results.append({'rule': p, 'x': p[0], 'y': p[1], 'net_profit': net_profit[0]})
    

    

In [ ]:
df = pd.DataFrame(results)# .sort_values('net_profit')

In [ ]:
plt.scatter(df.x, df.y, s = 100,  c = df.net_profit, cmap = 'rainbow')
c = plt.colorbar()
#fig = plt.figure()
#ax = fig.gca(projection='3d')
##ax = fig.add_subplot(111, projection='3d')
#ax.plot_surface(df.x, df.y, df.net_profit)

In [ ]:
df.sort_values('net_profit').tail()

In [ ]:
import numpy as np

class GeneticGeneric:
    def __init__(self, strategy_instance, params_list, **kwargs):
        random.seed(kwargs.get('rand_seed', None))
        
        self.strategy = strategy_instance
        self.params_list = params_list
        self.params_universe = list(itertools.product(*[range(len(x[1])) for x in params_list]))
        self.params_weights = [np.zeros((len(x[1]), 2)) for x in self.params_list]
        
    def mutate(self, individual, params_uni):
        rnd_gene_idx = random.randint(0, len(individual)-1)        
        new_gene = random.choice(params_uni)        

        individual[rnd_gene_idx] = new_gene[rnd_gene_idx]

        return individual,
    
    def mate(self, ind1, ind2):
        return tools.cxOnePoint(ind1, ind2)
    
    def mate_strongest(self, ind1, ind2, pbstrongest=0.6):
        def gene_weight(param_idx, gene_idx):
            score_sum, score_cnt = self.params_weights[param_idx][gene_idx]
            if score_cnt == 0:
                return 0.0
            else:
                return score_sum/score_cnt
        
        for param_i, genes in enumerate(zip(ind1, ind2)):            
            gene_ind1 = genes[0]
            gene_ind2 = genes[1]
            
            if random.random() <= pbstrongest:
                w1 = gene_weight(param_i, gene_ind1)                
                w2 = gene_weight(param_i, gene_ind2)
                
                if w1 < w2:
                    # Swap genes if ind2 has stronger gene than ind1
                    tmp_gene = gene_ind1
                    ind1[param_i] = gene_ind2
                    ind2[param_i] = tmp_gene
                    
        return ind1, ind2        
        
    
    def evaluate(self, individual):
        args = [self.params_list[i][1][p] for i,p in enumerate(individual)]        
        strategy_exposure = self.strategy.calculate(*args)
        score = score_netprofit(self.strategy.dm.quotes()['c'].values, strategy_exposure.values)
        
        # Updating the weights
        for i, p in enumerate(individual):
            self.params_weights[i][p][0] += score
            self.params_weights[i][p][1] += 1.0
            
        
        return (score,)
    
    def run(self):
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMax)


        toolbox = base.Toolbox()
        toolbox.register("rules", random.choice, self.params_universe)
        toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.rules)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)


        toolbox.register("evaluate", self.evaluate)
        #toolbox.register("mate", self.mate)
        toolbox.register("mate", self.mate_strongest)
        toolbox.register("mutate", self.mutate,  params_uni=self.params_universe)
        toolbox.register("select", tools.selTournament, tournsize=5)
        
        pop = toolbox.population(n=200)
        
        stats = tools.Statistics(key=lambda ind: ind.fitness.values)
        
        stats.register("avg", np.mean)
        stats.register("std", np.std)
        stats.register("min", np.min)
        stats.register("max", np.max)
        #stats.register("cnt", len)

        self.hof = deap.tools.HallOfFame(10)
        self.pop, self.logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, 
                             ngen=50, verbose=True, stats=stats, halloffame=self.hof)
        
    def plot(self):
        gen = self.logbook.select("gen")
        fit_mins = self.logbook.select("min")
        size_avgs = self.logbook.select("avg")

        fig, ax1 = plt.subplots()
        line1 = ax1.plot(gen, fit_mins, "b-", label="Minimum Fitness")
        ax1.set_xlabel("Generation")
        ax1.set_ylabel("Fitness", color="b")
        for tl in ax1.get_yticklabels():
            tl.set_color("b")

        ax2 = ax1.twinx()
        line2 = ax2.plot(gen, size_avgs, "r-", label="Average Fitness")
        ax2.set_ylabel("Size", color="r")
        for tl in ax2.get_yticklabels():
            tl.set_color("r")

        lns = line1 + line2
        labs = [l.get_label() for l in lns]
        ax1.legend(lns, labs, loc="lower right", frameon=True)

        plt.show()

In [ ]:
[(i, z) for i,z in enumerate(zip([1,2, 3], [3, 4, 5]))]

In [ ]:
gen = GeneticGeneric(a, PARAMS) #, rand_seed=10)

In [ ]:

gen.run()

In [ ]:
for i, pwght in enumerate(gen.params_weights):
    param_name = gen.params_list[i][0]
    
    avg = pwght[:, 0] / pwght[:, 1]
    #print(avg)
    
    fig, ax = plt.subplots()
    rects1 = ax.bar(gen.params_list[i][1], avg, 0.35, color='r')
    
    ax.set_ylabel('Scores')
    ax.set_title(f'Mean scores by {param_name}')
    

# TODO: hall of fame optimization

In [ ]:
gen.plot()


In [ ]:
[x for x in gen.hof]

In [ ]:
gen.pop

In [ ]:
import calendar

In [ ]:
calendar.monthcalendar(2017, 1)

In [ ]:
from datetime import datetime

In [ ]:
datetime.now().time()

In [ ]:
datetime.combine()

In [ ]:
from dateutil.rrule import rrule, MONTHLY, WEEKLY
from datetime import datetime
import pytz 
tz = pytz.UTC


start_date = tz.localize(datetime(2017, 1, 1, 12, 45))
list(rrule(freq=MONTHLY, count=4, dtstart=start_date + relativedelta(months=11), bysetpos=-1, byweekday=5, interval=12))

In [ ]:
from dateutil.relativedelta import relativedelta

In [ ]:
start_date = tz.localize(datetime(2017, 1, 1, 12, 45)).date()
end_date = tz.localize(datetime(2017, 5, 28, 12, 45)).date()

d = relativedelta(end_date, start_date)

In [ ]:
d.months + 2

In [ ]:
start_date + relativedelta(month=4)